In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys 
import time
import pandas as pd
import random
import re

#기간(2023.01.01 ~)부터인것만 수집
browser = webdriver.Chrome()
url = 'https://www.youtube.com/@SamsungKorea/videos'
browser.get(url)

results = []
for i in range(10):
    # 화면 빈 곳을 선택
    body = browser.find_elements('css selector', 'body')[0]
    # Page Down 버튼을 입력..
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(random.uniform(2, 4))

    html = browser.page_source 
    soup = BeautifulSoup(html, 'html.parser')
    video_list = soup.select('ytd-rich-grid-media.style-scope.ytd-rich-item-renderer')

    for video in video_list:
        title = video.select('#video-title')[0].text

        view = video.select('div#metadata-line > span')[0].text
        numbers = re.findall(r'\d+', view)
        view_number = int(numbers[0])

        original_date = video.select('div#metadata-line > span')[1].text
        date = original_date
        # '일 전', '개월 전', '년 전'에 따라 다르게 처리
        if '시간 전' in date:
            date = 1 
        elif '일 전' in date:
            date = int(date.replace('일 전', ''))
        elif '주 전' in date:
            date = int(date.replace('주 전', '')) * 3
        elif '개월 전' in date:
            date = int(date.replace('개월 전', '')) * 30
        elif '년 전' in date:
            date = int(date.replace('년 전', '')) * 365
        else:
            try:
                date = int(date)
            except ValueError:
                print(f"{date}를 숫자로 변환할 수 없습니다.")
                continue
                
         # 날짜가 330 이하인 경우만 수집
        if date <= 330 and '갤럭시' in title: 
    #          print(title, '|', original_date, '|', view_number,'|',full_link)
            data = [title, original_date, view]
            results.append(data)

df = pd.DataFrame(results)
df.columns = ['title','original_date','view']

df.to_csv('삼성전자 갤럭시 광고 정보들.csv', index=False)